In [2]:
import os
import dlib
import cv2
import numpy as np
import tensorflow as tf
from keras_facenet import FaceNet
from pymongo import MongoClient
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime

In [ ]:
class FaceRecognitionSystem:
    def __init__(self, config):
        # Initialize paths
        self.BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
        self.DATA_DIR = os.path.join(self.BASE_DIR, 'data')
        
        # Initialize models
        self.embedder = FaceNet()
        self.detector = dlib.get_frontal_face_detector()
        
        # Database setup
        self.mongodb_uri = config.get('mongodb_uri')
        self.client = MongoClient(self.mongodb_uri)
        self.db = self.client.face_recognition
        self.collection = self.db.embeddings
        
        # Recognition parameters
        self.recognition_threshold = config.get('recognition_threshold')
        
        # Initialize attendance sheet
        self.attendance = pd.DataFrame(columns=["Name", "Time", "Date", "Status"])

    def extract_face(self, image_path):
        """Extract and preprocess face from image path"""
        img = cv2.imread(image_path)
        if img is None:
            return None
            
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        faces = self.detector(rgb_img, 1)
        
        if len(faces) == 0:
            return None
            
        try:
            face = faces[0]
            x1, y1 = max(face.left(), 0), max(face.top(), 0)
            x2, y2 = min(face.right(), rgb_img.shape[1]), min(face.bottom(), rgb_img.shape[0])
            
            if x2 <= x1 or y2 <= y1:
                return None
                
            face_region = rgb_img[y1:y2, x1:x2]
            return cv2.resize(face_region, (160, 160))
        except:
            return None

    def store_embeddings(self):
        total_images = 0
        success_count = 0
        failed_images = []
        skipped_persons = 0
    
        person_dirs = [d for d in os.listdir(self.DATA_DIR) 
                      if os.path.isdir(os.path.join(self.DATA_DIR, d))]
        
        for person_name in person_dirs:
            # Check if person already exists in database
            if self.collection.count_documents({"person_name": person_name}) > 0:
                print(f"⏩ Skipping {person_name} - already exists in database")
                skipped_persons += 1
                continue
    
            person_dir = os.path.join(self.DATA_DIR, person_name)
            print(f"\nProcessing: {person_name}")
            
            image_files = [f for f in os.listdir(person_dir) 
                          if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            
            person_embeddings = []
            for image_file in image_files:
                total_images += 1
                image_path = os.path.join(person_dir, image_file)
                face = self.extract_face(image_path)
                
                if face is not None:
                    try:
                        face = face.astype('float32') / 255.0
                        face = np.expand_dims(face, axis=0)
                        
                        embedding = self.embedder.model.predict(face, verbose=0)[0]
                        embedding /= np.linalg.norm(embedding)
                        person_embeddings.append(embedding.tolist())
                        success_count += 1
                        
                    except Exception as e:
                        failed_images.append(image_path)
                else:
                    failed_images.append(image_path)
            
            # Insert all embeddings at once for new person
            if person_embeddings:
                self.collection.insert_one({
                    "person_name": person_name,
                    "embeddings": person_embeddings
                })
    
        print("\n✅ Processing Complete!")
        print(f"Total Persons Processed: {len(person_dirs) - skipped_persons}")
        print(f"Skipped Existing Persons: {skipped_persons}")
        print(f"Total Images Processed: {total_images}")
        print(f"Successful Embeddings: {success_count}")
        print(f"Failed Images: {len(failed_images)}")
        
    def recognize_faces(self):
        """Real-time face recognition from webcam"""
        cap = cv2.VideoCapture(0)
        cap.set(3, 640)  # Width
        cap.set(4, 480)  # Height
        
        # Load known embeddings
        known_faces = list(self.collection.find())
        names = []
        embeddings = []
        
        for face in known_faces:
            person_name = face['person_name']
            if 'embeddings' in face:
                for embedding in face['embeddings']:
                    names.append(person_name)
                    embeddings.append(embedding)
        
        embeddings = np.array(embeddings)
        
        while True:
            success, frame = cap.read()
            if not success:
                break
            
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            faces = self.detector(rgb)
            
            for face in faces:
                x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
                face_img = rgb[y1:y2, x1:x2]
                
                if face_img.size == 0:
                    continue
                
                resized = cv2.resize(face_img, (160, 160))
                embedding = self.embedder.embeddings([resized])[0]
                embedding /= np.linalg.norm(embedding)
                
                similarities = np.dot(embeddings, embedding)
                best_match_idx = np.argmax(similarities)
                best_similarity = similarities[best_match_idx]
                
                # Recognition logic
                if best_similarity > self.recognition_threshold:
                    name = names[best_match_idx]
                    status = "Present"
                    color = (0, 255, 0)  # Green for known faces
                else:
                    name = "Unknown"
                    status = "Unknown"
                    color = (0, 0, 255)  # Red for unknown faces
                
                # Update attendance only once per session
                if name not in self.attendance.Name.values:
                    now = datetime.now()
                    self.attendance.loc[len(self.attendance)] = [
                        name,
                        now.strftime("%H:%M:%S"),
                        now.strftime("%Y-%m-%d"),
                        status
                    ]
                
                # Draw bounding box and label
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, name, (x1, y1-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
            
            cv2.imshow('Face Recognition', frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        
    def mark_absentees(self):
        """Generate attendance report with absentees marked in a single consolidated file"""
        registered = self.collection.distinct("person_name")
        today = datetime.now().date().strftime("%Y-%m-%d")
        
        # Use in-memory attendance data instead of reading from Excel
        present = self.attendance[self.attendance["Date"] == today]["Name"].unique().tolist()
        
        # Generate full attendance record for today
        full_record = []
        for person in registered:
            status = "Present" if person in present else "Absent"
            full_record.append({
                "Name": person,
                "Date": today,
                "Time": datetime.now().strftime("%H:%M:%S"),
                "Status": status
            })
        
        # Create DataFrame for today's records
        df_today = pd.DataFrame(sorted(full_record, key=lambda x: x["Name"]))
        
        # Define consolidated filename
        consolidated_file = "../attendance_data/attendance_records.xlsx"
        
        # Append to existing file or create new
        if os.path.exists(consolidated_file):
            existing_df = pd.read_excel(consolidated_file)
            updated_df = pd.concat([existing_df, df_today], ignore_index=True)
        else:
            updated_df = df_today
        
        # Save consolidated data
        updated_df = updated_df.drop_duplicates(subset=["Name", "Date"], keep="last")
        updated_df.to_excel(consolidated_file, index=False)
            

In [4]:
config = {
    "mongodb_uri": "mongodb://localhost:27017/face-recognition",
    "recognition_threshold": 0.75
}

In [5]:
frs = FaceRecognitionSystem(config)

# Store embeddings from images
frs.store_embeddings()





⏩ Skipping 2_Dipesh_Bajgain - already exists in database
⏩ Skipping 4_diense - already exists in database
⏩ Skipping 8_dinesh - already exists in database
⏩ Skipping 4_dinesh - already exists in database
⏩ Skipping 9_Milan_Bajgain - already exists in database
⏩ Skipping 4_Dinesh_Bajgain - already exists in database
⏩ Skipping 8_Harendra_sir - already exists in database
⏩ Skipping 3_Ayudh_Pantha - already exists in database
⏩ Skipping 6_Dpt_Prjuli - already exists in database
⏩ Skipping 18_SRK - already exists in database
⏩ Skipping 10_Bibek_Adhikari - already exists in database

✅ Processing Complete!
Total Persons Processed: 0
Skipped Existing Persons: 11
Total Images Processed: 0
Successful Embeddings: 0
Failed Images: 0


In [6]:
# Run real-time recognition
frs.recognize_faces()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 748ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━

In [7]:
# Generate attendance report
frs.mark_absentees()